In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Opprette klienten

I dette eksempelet skal vi bruke [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) for tilgang til LLM.

`model` er definert som `gpt-4o-mini`. Prøv å endre modellen til en annen modell som er tilgjengelig på GitHub Models-markedet for å se forskjellige resultater.

Som en rask test skal vi bare kjøre en enkel forespørsel - `Hva er hovedstaden i Frankrike`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definere funksjonene

I dette eksempelet skal vi gi agenten tilgang til et verktøy som er en funksjon med en liste over tilgjengelige feriedestinasjoner og deres tilgjengelighet.

Du kan tenke deg at dette kunne vært et scenario der en reiseagent har tilgang til en reisekatalog eller database, for eksempel.

Mens du går gjennom dette eksempelet, kan du gjerne prøve å definere nye funksjoner og verktøy som agenten kan bruke.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Definere funksjonsverktøyet 
For at agenten skal kunne bruke `vacation_destinations` som et `FunctionTool`, må vi definere det som et slikt verktøy. 

Vi vil også gi en beskrivelse av verktøyet, som hjelper agenten med å forstå hva verktøyet brukes til i forhold til oppgaven brukeren har bedt om.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Definere agenten

Nå kan vi opprette agenten i koden nedenfor. Vi definerer `system_message` for å gi agenten instruksjoner om hvordan den skal hjelpe brukere med å finne feriedestinasjoner.

Vi setter også parameteren `reflect_on_tool_use` til true. Dette gjør at LLM kan ta svaret fra verktøykallet og sende svaret ved hjelp av naturlig språk.

Du kan sette parameteren til false for å se forskjellen.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Kjøre agenten

Nå kan vi kjøre agenten med den første brukermeldingen som spør om å ta en tur til Tokyo.

Du kan endre denne destinasjonsbyen for å se hvordan agenten reagerer på tilgjengeligheten av byen.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Ansvarsfraskrivelse**:  
Dette dokumentet er oversatt ved hjelp av AI-oversettelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selv om vi streber etter nøyaktighet, vær oppmerksom på at automatiserte oversettelser kan inneholde feil eller unøyaktigheter. Det originale dokumentet på sitt opprinnelige språk bør anses som den autoritative kilden. For kritisk informasjon anbefales profesjonell menneskelig oversettelse. Vi er ikke ansvarlige for misforståelser eller feiltolkninger som oppstår ved bruk av denne oversettelsen.
